# Requests 和 正则表达式 爬取猫眼电影榜单

In [1]:
import requests
import re

In [3]:
headers = {
    'Referer':'https://maoyan.com/board',
    'Origin': 'https://maoyan.com',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36 Edge/18.18362',
    'Accept-Encoding': 'zip, deflate, br'
}
response = requests.get('https://maoyan.com/board/4?offset=0', headers=headers)
print(response.status_code)
with open('demo_file/maoyan.txt','w', encoding='utf-8') as f:
    f.write(response.text)

200


In [10]:
test = '''
<dd>
                        <i class="board-index board-index-1">1</i>
    <a href="/films/1203" title="霸王别姬" class="image-link" data-act="boarditem-click" data-val="{movieId:1203}">
      <img src="//s3plus.meituan.net/v1/mss_e2821d7f0cfe4ac1bf9202ecf9590e67/cdn-prod/file:5788b470/image/loading_2.e3d934bf.png" alt="" class="poster-default" />
      <img data-src="https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c" alt="霸王别姬" class="board-img" />
    </a>
    <div class="board-item-main">
      <div class="board-item-content">
              <div class="movie-item-info">
        <p class="name"><a href="/films/1203" title="霸王别姬" data-act="boarditem-click" data-val="{movieId:1203}">霸王别姬</a></p>
        <p class="star">
                主演：张国荣,张丰毅,巩俐
        </p>
<p class="releasetime">上映时间：1993-01-01</p>    </div>
    <div class="movie-item-number score-num">
<p class="score"><i class="integer">9.</i><i class="fraction">5</i></p>        
    </div>

      </div>
    </div>

                </dd>
'''

In [16]:
movie_pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*?<a.*?image-link.*?<img data-src="(.*?)".*?class="board-img" />.*?class="name"><a href="(.*?)".*?>(\w+)</a></p>.*?<p class="star">\s+(.*?)\s+</p>.*?<p class="releasetime">(.*?)</p>.*?<i class="integer">(\d)\.</i>.*?fraction">(\d)</i></p>.*?</dd>', re.S)
# movie_pattern = re.compile('<dd>.*?board-index.*?>(\d+)</i>.*data-src="(.*?)".*?name">')
result = re.findall(movie_pattern, response.text)
print(result)

[('1', 'https://p1.meituan.net/movie/20803f59291c47e1e116c11963ce019e68711.jpg@160w_220h_1e_1c', '/films/1203', '霸王别姬', '主演：张国荣,张丰毅,巩俐', '上映时间：1993-01-01', '9', '5'), ('2', 'https://p0.meituan.net/movie/283292171619cdfd5b240c8fd093f1eb255670.jpg@160w_220h_1e_1c', '/films/1297', '肖申克的救赎', '主演：蒂姆·罗宾斯,摩根·弗里曼,鲍勃·冈顿', '上映时间：1994-09-10(加拿大)', '9', '5'), ('3', 'https://p0.meituan.net/movie/289f98ceaa8a0ae737d3dc01cd05ab052213631.jpg@160w_220h_1e_1c', '/films/2641', '罗马假日', '主演：格利高里·派克,奥黛丽·赫本,埃迪·艾伯特', '上映时间：1953-09-02(美国)', '9', '1'), ('4', 'https://p1.meituan.net/movie/6bea9af4524dfbd0b668eaa7e187c3df767253.jpg@160w_220h_1e_1c', '/films/4055', '这个杀手不太冷', '主演：让·雷诺,加里·奥德曼,娜塔莉·波特曼', '上映时间：1994-09-14(法国)', '9', '5'), ('5', 'https://p1.meituan.net/movie/b607fba7513e7f15eab170aac1e1400d878112.jpg@160w_220h_1e_1c', '/films/267', '泰坦尼克号', '主演：莱昂纳多·迪卡普里奥,凯特·温丝莱特,比利·赞恩', '上映时间：1998-04-03', '9', '5'), ('6', 'https://p0.meituan.net/movie/da64660f82b98cdc1b8a3804e69609e041108.jpg@160w_220h_1e_1c', '/films